##### Copyright 2019 The TensorFlow Authors.

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
columns = ['open', '3before', '6before', '3after', 'goodbuy']
shouldbuy = ['do-not-buy', 'buy', 'great-buy']

In [ ]:


training_file = tf.keras.utils.get_file(
    "lite30.csv","https://raw.githubusercontent.com/jim-cassidy/data/main/lite30.csv")
testing_file = tf.keras.utils.get_file(
    "iris_test.csv", "https://raw.githubusercontent.com/jim-cassidy/data/main/iris_test.csv")

train = pd.read_csv(training_file, names=columns, header=0)
test = pd.read_csv(testing_file, names=columns, header=0)

In [ ]:
train_y = train.pop('goodbuy')
test_y = test.pop('goodbuy')


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # make dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
# Feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
# Make DNN
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppmjb_k4h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Train!!
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=15000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpeedh05cc/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 62.195396, step = 0
INFO:tensorflow:global_step/sec: 422.832
INFO:tensorflow:loss = 33.98441, step = 100 (0.242 sec)
INFO:tensorflow:global_step/sec: 514.775
INFO:tensorflow:loss = 24.766697, step = 200 (0.193 sec)
INFO:tensorflow:global_step/sec: 547.289
INFO:tensorflow:loss = 19.049324, step = 300 (0.183 sec)
INFO:tensorflow:global_step/sec: 557.118
INFO:tensorflow:loss = 14.4170885, step = 400 (0.177 sec)
INFO:tensorflow:global_step/sec: 535.414
INFO:tensorflow:loss = 11.681767, step = 500 (0.189 sec)
INFO:t

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-15T22:16:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpeedh05cc/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.19605s
INFO:tensorflow:Finished evaluation at 2021-10-15-22:16:07
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.26666668, average_loss = 1.2726277, global_step = 15000, loss = 1.2726277
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /tmp/tmpeedh05cc/model.ckpt-15000

Test accuracy: 0.267



In [ ]:

# Generate predictions 
expected = ['do-not-buy', 'buy', 'great-buy']
predict_x = {
    'open': [55],
    '3before': [55],
    '6before': [46],
    '3after': [129],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [ ]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        shouldbuy[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpeedh05cc/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "buy" (100.0%), expected "do-not-buy"
